In [1]:
# Vadana Ioan-Grigore
# 334AA

import tkinter as tk
from tkinter import ttk, messagebox
import json
import cv2
import pyaudio
import wave
import speech_recognition as sr
import os
import socket
import numpy as np

root = tk.Tk()
root.title("Aplicație de Securitate")

notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill='both')

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

def load_from_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

users_file = 'users.json'
users = load_from_json(users_file)

password_file = 'parole.txt'

current_user = None

def register_user():
    global current_user
    nume = entry_nume.get()
    prenume = entry_prenume.get()
    parola = entry_parola.get()

    if not nume or not prenume or not parola:
        messagebox.showerror("Eroare", "Toate câmpurile sunt obligatorii!")
        return
    
    for user in users:
        if user['nume'] == nume and user['prenume'] == prenume:
            messagebox.showerror("Eroare", "Utilizatorul există deja!")
            return
    
    new_user = {
        "nume": nume,
        "prenume": prenume,
        "parola": parola
    }
    users.append(new_user)
    save_to_json(users, users_file)
    messagebox.showinfo("Succes", "Utilizatorul a fost înregistrat cu succes!")
    current_user = new_user

    entry_nume.delete(0, tk.END)
    entry_prenume.delete(0, tk.END)
    entry_parola.delete(0, tk.END)

    notebook.tab(1, state="normal")
    notebook.tab(2, state="normal")
    notebook.tab(3, state="normal")

tab1 = ttk.Frame(notebook)
notebook.add(tab1, text="Înregistrare Utilizator")

tab2 = ttk.Frame(notebook)
notebook.add(tab2, text="Înregistrare Video Utilizator")

tab3 = ttk.Frame(notebook)
notebook.add(tab3, text="Înregistrare Audio Utilizator")

tab4 = ttk.Frame(notebook)
notebook.add(tab4, text="Logare Server")

notebook.tab(1, state="disabled")
notebook.tab(2, state="disabled")
notebook.tab(3, state="disabled")

{}

In [2]:
label_nume = ttk.Label(tab1, text="Nume:")
label_nume.grid(column=0, row=0, padx=10, pady=10)
entry_nume = ttk.Entry(tab1)
entry_nume.grid(column=1, row=0, padx=10, pady=10)

label_prenume = ttk.Label(tab1, text="Prenume:")
label_prenume.grid(column=0, row=1, padx=10, pady=10)
entry_prenume = ttk.Entry(tab1)
entry_prenume.grid(column=1, row=1, padx=10, pady=10)

label_parola = ttk.Label(tab1, text="Parola:")
label_parola.grid(column=0, row=2, padx=10, pady=10)
entry_parola = ttk.Entry(tab1, show="*")
entry_parola.grid(column=1, row=2, padx=10, pady=10)

button_register = ttk.Button(tab1, text="Înregistrare Utilizator", command=register_user)
button_register.grid(column=0, row=3, columnspan=2, pady=10)

In [3]:
def record_video():
    global current_user
    if not current_user:
        messagebox.showerror("Eroare", "Nu există utilizator înregistrat!")
        return

    messagebox.showinfo("Informație", "Înregistrarea video va începe. Asigurați-vă că sunteți pregătit.")
    cap = cv2.VideoCapture(0)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_filename = f"user_video_{current_user['nume'].lower()}_{current_user['prenume'].lower()}.avi"
    out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))

    for _ in range(60):  # 3 secunde la 20 fps
        ret, frame = cap.read()
        if ret:
            out.write(frame)
            cv2.imshow('Recording...', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Succes", "Înregistrarea video a fost efectuată cu succes!")

    notebook.tab(2, state="normal")

button_record_video = ttk.Button(tab2, text="Înregistrare Video Utilizator", command=record_video)
button_record_video.pack(pady=20)

In [4]:
def record_audio():
    global current_user
    if not current_user:
        messagebox.showerror("Eroare", "Nu există utilizator înregistrat!")
        return

    messagebox.showinfo("Informație", "Înregistrarea audio va începe. Vă rugăm să vorbiți clar.")
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    audio_filename = f"user_audio_{current_user['nume'].lower()}_{current_user['prenume'].lower()}.wav"
    text_filename = f"user_audio_text_{current_user['nume'].lower()}_{current_user['prenume'].lower()}.txt"

    try:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source, timeout=5)
        
        with open(audio_filename, "wb") as f:
            f.write(audio.get_wav_data())

        try:
            text = recognizer.recognize_google(audio)
            with open(text_filename, "w") as f:
                f.write(text)
            messagebox.showinfo("Succes", f"Înregistrarea audio a fost efectuată cu succes! Text: {text}")
        except sr.UnknownValueError:
            messagebox.showerror("Eroare", "Nu am putut înțelege ce ați spus!")
        except sr.RequestError as e:
            messagebox.showerror("Eroare", f"Eroare de recunoaștere vocală; {e}")
    except Exception as e:
        messagebox.showerror("Eroare", f"A apărut o eroare în timpul înregistrării audio: {e}")

    notebook.tab(3, state="normal")

button_record_audio = ttk.Button(tab3, text="Înregistrare Audio Utilizator", command=record_audio)
button_record_audio.pack(pady=20)

In [5]:
def authenticate_user():
    global current_user
    if not current_user:
        messagebox.showerror("Eroare", "Nu există utilizator înregistrat!")
        return

    # Recunoaștere facială
    video_filename = f"user_video_{current_user['nume'].lower()}_{current_user['prenume'].lower()}.avi"
    if not os.path.exists(video_filename):
        messagebox.showerror("Eroare", "Fișierul video nu există!")
        return

    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Încarcă videoclipul înregistrat și pregătește recunoașterea facială
    cap_video = cv2.VideoCapture(video_filename)
    frames = []
    while cap_video.isOpened():
        ret, frame = cap_video.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)
    cap_video.release()

    if not frames:
        messagebox.showerror("Eroare", "Videoclipul nu conține cadre!")
        return

    print(f"Număr cadre în videoclip: {len(frames)}")

    labels = np.zeros(len(frames), dtype=int)

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(frames, labels)

    print("Antrenare finalizată")

    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Eroare", "Nu am putut accesa camera!")
        return
    cap.release()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    print(f"Număr fețe detectate: {len(faces)}")

    for (x, y, w, h) in faces:
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])
        print(f"ID: {id}, Încredere: {confidence}")  # Debug: Afișează ID și încrederea
        if confidence < 90:  # Pragul de încredere ajustat
            messagebox.showinfo("Succes", "Autentificare video reușită, trecem la autentificarea vocală.")
            break
    else:
        messagebox.showerror("Eroare", "Recunoașterea facială a eșuat!")
        return

    # Recunoaștere vocală
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source, timeout=5)

    try:
        text = recognizer.recognize_google(audio)
        # Debug: Afișează textul recunoscut
        print(f"Text recunoscut: {text}")

        if text != current_user['parola']:
            messagebox.showerror("Eroare", "Parola vocală nu se potrivește cu parola introdusă!")
            return

        found = False
        with open(password_file, "r") as f:
            passwords = f.readlines()
        for line in passwords:

            print(f"Linie din fișierul parole: {line.strip()}")
            name, surname, password = line.strip().split(',')
            if name == current_user['nume'] and surname == current_user['prenume'] and password == text:
                messagebox.showinfo("Succes", "Autentificare reușită!")
                send_message_to_server(current_user)
                found = True
                break
        if not found:
            messagebox.showerror("Eroare", "Parola vocală nu se regăsește în fișierul de parole!")
    except sr.UnknownValueError:
        messagebox.showerror("Eroare", "Nu am putut înțelege ce ați spus!")
    except sr.RequestError as e:
        messagebox.showerror("Eroare", f"Eroare de recunoaștere vocală; {e}")

def send_message_to_server(user):
    # Inițializare conexiune la server
    server_ip = '127.0.0.1'
    server_port = 65432

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((server_ip, server_port))
            message = f"Utilizatorul {user['nume']} {user['prenume']} s-a conectat la server."
            s.sendall(message.encode('utf-8'))
            data = s.recv(1024)
            print('Received', repr(data))
            messagebox.showinfo("Succes", "Conectarea la server a reușit!")
    except ConnectionRefusedError:
        messagebox.showerror("Eroare", "Conexiunea la server a eșuat!")
    except Exception as e:
        messagebox.showerror("Eroare", f"Eroare de conectare la server: {e}")

# Buton pentru autentificare
button_authenticate = ttk.Button(tab4, text="Logare Server", command=authenticate_user)
button_authenticate.pack(pady=20)

In [6]:
# Startăm bucla principală a interfeței grafice
root.mainloop()

Număr cadre în videoclip: 60
Antrenare finalizată
Număr fețe detectate: 1
ID: 0, Încredere: 76.33081868531937
Număr cadre în videoclip: 60
Antrenare finalizată
Număr fețe detectate: 1
ID: 0, Încredere: 72.82779647471055
Text recunoscut: animal
Număr cadre în videoclip: 60
Antrenare finalizată
Număr fețe detectate: 0
Număr cadre în videoclip: 60
Antrenare finalizată
Număr fețe detectate: 1
ID: 0, Încredere: 75.33181696332879
